In [ ]:
!pip install yfinance
!pip install yahooquery
from yahooquery import Screener
import yfinance as yf   
import pandas as pd
import numpy as np
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense,LSTM
from tensorflow.keras.layers import Dropout
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler,StandardScaler
import tensorflow as tf
import random 

In [14]:
clmns = [
    "Open 12-11","Open 11-10","Open 10-9","Open 9-8","Open 8-7","Open 7-6","Open 6-5","Open 5-4","Open 4-3","Open 3-2","Open 2-1",
    "Close 12-11","Close 11-10","Close 10-9","Close 9-8","Close 8-7","Close 7-6","Close 6-5","Close 5-4","Close 4-3","Close 3-2","Close 2-1",
    "High 12-11","High 11-10","High 10-9","High 9-8","High 8-7","High 7-6","High 6-5","High 5-4","High 4-3","High 3-2","High 2-1",
    "Low 12-11","Low 11-10","Low 10-9","Low 9-8","Low 8-7","Low 7-6","Low 6-5","Low 5-4","Low 4-3","Low 3-2","Low 2-1",
    "AdjV 12-11","AdjV 11-10","AdjV 10-9","AdjV 9-8","AdjV 8-7","AdjV 7-6","AdjV 6-5","AdjV 5-4","AdjV 4-3","AdjV 3-2","AdjV 2-1",
    "Volume 12-11","Volume 11-10","Volume 10-9","Volume 9-8","Volume 8-7","Volume 7-6","Volume 6-5","Volume 5-4","Volume 4-3","Volume 3-2","Volume 2-1",
    "suggestion"
]

def read_syms_from_txt():  
  with open("syms.txt","r") as f:
    lines = f.readlines()
    f.close()
  lst = []
  for line in lines:
    line = line.strip()
    lst.append(line)
  symbols = lst
  return symbols

def get_crypto_syms():
   #'all_cryptocurrencies_au','all_cryptocurrencies_ca','all_cryptocurrencies_eu','all_cryptocurrencies_gb','all_cryptocurrencies_in',
   screens = [
   'all_cryptocurrencies_us','all_cryptocurrencies_au','all_cryptocurrencies_ca','all_cryptocurrencies_eu','all_cryptocurrencies_gb','all_cryptocurrencies_in',]
   s = Screener()
   symbols = []
   for i in screens:
      data = s.get_screeners(i, count=250)
      dicts = data[i]['quotes']
      syms = [d['symbol'] for d in dicts]
      for sym in syms:
        symbols.append(sym)
   #print(len(symbols))
   #pieces = 15
   # new_arrays = np.array_split(symbols, pieces)
   return symbols

def spliting(data):
  X = data.drop(["suggestion"],axis=1)
  y = data["suggestion"]
  xTrain, xTest, yTrain, yTest = train_test_split(X, y, test_size=0.1)
  print(xTrain.shape,end=" ")
  print(yTrain.shape)
  print(xTest.shape,end=" ")
  print(yTest.shape)
  return xTrain, xTest, yTrain, yTest


def scaler(row):
    scaler = MinMaxScaler(feature_range=(-10, 10))
    row = scaler.fit_transform(row)
    return row

def process(data):
    data = data.dropna()
    row = []
    data = np.array(data)
    llst = [0, 1, 2, 3, 4, 5]

    for i in range(12, data.shape[0]):
        grow = []
        srow = []

        for j in llst:

           srow.append([
               data[i-1][j] - data[i-2][j],
                data[i-2][j] - data[i-3][j],
                data[i-3][j] - data[i-4][j],
                data[i-4][j] - data[i-5][j],
                data[i-5][j] - data[i-6][j],
                data[i-6][j] - data[i-7][j],
                data[i-7][j] - data[i-8][j],
                data[i-8][j] - data[i-9][j],
                data[i-9][j] - data[i-10][j],
                data[i-10][j] - data[i-11][j],
                data[i-11][j] - data[i-12][j] 
                 ])
           
           
        for lst in srow:
            mm = np.array(lst)
            mm = np.reshape(mm, (-1, 1))
            grow.append(scaler(mm))

        sugg = 0
        if data[i][3] > data[i][0]:
            sugg = 1
        
        arr = np.array(grow).flatten()
        arr = np.append(arr, sugg)
        row.append(arr)

    grow = []
    srow = []
    llst = []
    data = []
    arr = []
    mm = []
    return np.array(row)
         

In [ ]:
def get_data(symbols):
    unattached_dfs = []
    for symbol in symbols:
           data = yf.download(symbol,period="730d",interval="1h",progress=False)
           if data.empty :
              pass
           else:
               if data.shape[0] > 12:
                   unattached_dfs.append(pd.DataFrame(process(data),columns=clmns))
    symbols = []
    symbol = []
    data = []

    data = pd.concat(unattached_dfs)
    data = data.astype(float)
    data = data.astype({'suggestion':'int'})

    xTrain, xTest, yTrain, yTest = spliting(data)
    print(data.shape)
    data.to_csv(f"{random.randint(1,1000)}.csv")
    return xTrain, xTest, yTrain, yTest,data

#symbols = ["MSFT","AAPL","GOOG","TSLA","AMZN"]
#symbols = ["BTC-USD","LTC-USD","TRX-USD","XRP-USD","ETH-USD","BNB-USD","DASH-USD","VET-USD","LINK-USD","ADA-USD","DOT-USD","SOL-USD","BCH-USD","FTT-USD","FIL-USD","XMR-USD"]
#symbols = ["AAPL","MSFT","TSLA","GOOG"]
#symbols = ["BTC-USD","ETH-USD"]
#symbols = ["BTC-USD"]
#symbols = read_syms_from_txt()
symbols = get_crypto_syms()

pieces = 10
new_arrays = np.array_split(symbols, pieces)
for symbols in new_arrays:
   get_data(symbols)
    
#xTrain, xTest, yTrain, yTest,data= get_data(symbols)
#data

In [22]:
data = pd.read_parquet("/content/drive/MyDrive/mio.parquet")
data = data.drop("Unnamed: 0",axis=1)
xTrain, xTest, yTrain, yTest= spliting(data)
data

(5212256, 55) (5212256,)
(579140, 55) (579140,)


,Open 12-11,Open 11-10,Open 10-9,Open 9-8,Open 8-7,Open 7-6,Open 6-5,Open 5-4,Open 4-3,Open 3-2,...,Volume 10-9,Volume 9-8,Volume 8-7,Volume 7-6,Volume 6-5,Volume 5-4,Volume 4-3,Volume 3-2,Volume 2-1,suggestion
0,-8.506937,-10.000000,-1.381924,10.000000,-2.812063,-5.432312,-2.648493,-4.313093,-5.151592,0.125634,...,-1.099808,-10.000000,10.000000,-0.103247,3.603015,0.479992,-0.086880,0.149994,-1.465175,0
1,3.919810,-8.506937,-10.000000,-1.381924,10.000000,-2.812063,-5.432312,-2.648493,-4.313093,-5.151592,...,0.479992,-1.099808,-10.000000,10.000000,-0.103247,3.603015,0.479992,-0.086880,0.149994,1
2,-6.972613,3.919810,-8.506937,-10.000000,-1.381924,10.000000,-2.812063,-5.432312,-2.648493,-4.313093,...,4.682885,0.479992,-1.099808,-10.000000,10.000000,-0.103247,3.603015,0.479992,-0.086880,0
3,-0.126968,-6.972613,3.919810,-8.506937,-10.000000,-1.381924,10.000000,-2.812063,-5.432312,-2.648493,...,-3.722901,4.682885,0.479992,-1.099808,-10.000000,10.000000,-0.103247,3.603015,0.479992,0
4,-9.492465,-0.126968,-6.972613,3.919810,-8.506937,-10.000000,-1.381924,10.000000,-2.812063,-5.432312,...,0.479992,-3.722901,4.682885,0.479992,-1.099808,-10.000000,10.000000,-0.103247,3.603015,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1804425,0.981812,9.356153,7.399614,10.000000,0.683214,0.652705,-2.491733,-7.769270,-10.000000,-3.103102,...,10.000000,7.330489,2.443496,2.443496,2.443496,2.443496,2.443496,2.395599,2.491394,1
1804426,-1.593182,0.981812,9.356153,7.399614,10.000000,0.683214,0.652705,-2.491733,-7.769270,-10.000000,...,-10.000000,10.000000,7.330489,2.443496,2.443496,2.443496,2.443496,2.443496,2.395599,1
1804427,-0.062593,-1.593182,0.981812,9.356153,7.399614,10.000000,0.683214,0.652705,-2.491733,-7.769270,...,1.844702,-10.000000,9.037578,6.496527,1.844702,1.844702,1.844702,1.844702,1.844702,0
1804428,-3.011133,-1.325838,-3.048574,-0.150318,9.275325,7.073165,10.000000,-0.486403,-0.520742,-4.059928,...,-4.137062,-4.137062,-10.000000,-0.576703,-1.834482,-4.137062,-4.137062,-4.137062,-4.137062,1


In [ ]:
model = Sequential()



model.add(Dense(2000, activation='relu', input_shape=(xTrain.shape[1],)))
model.add(Dense(1000, activation='relu'))
model.add(Dense(1000, activation='relu'))
model.add(Dense(500, activation='relu'))

model.add(Dense(1, activation='sigmoid'))
model.compile(optimizer="adam", loss='binary_crossentropy',metrics=['accuracy'])
model.summary()

In [ ]:
model.fit(xTrain,yTrain,epochs=50,batch_size=8192,validation_data=(xTest,yTest))

In [ ]:
model.save("fffirst_model.h5")
model.fit(xTrain,yTrain,epochs=10,batch_size=256,validation_data=(xTest,yTest))
model.save("sssecond_model.h5")

In [18]:
from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive
